In [1]:
import sys
import os
import re
sys.path.append('bert')
from pathlib import Path

from run_classifier import *

bert_config_file = 'cased_L-12_H-768_A-12/bert_config.json'
bert_config = modeling.BertConfig.from_json_file(bert_config_file)
output_dir = 'experiments/test'
learning_rate = 2e-5
num_train_steps = None
num_warmup_steps = None
train_batch_size = 32
eval_batch_size = 8
predict_batch_size = 8
save_checkpoints_steps = 1000
iterations_per_loop = 1000
num_tpu_cores = 8
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
# data_dir = 'experiments/general/store/geosciences/'
processor = STIProcessor()
label_list = processor.get_labels()
vocab_file = 'cased_L-12_H-768_A-12/vocab.txt'
do_lower_case = True

In [4]:
p = Path('experiments/general/store')
terms = [x.stem for x in p.iterdir() if x.is_dir() if x.stem != 'lost+found']

In [5]:
print(len(terms))
terms

11


['space_sciences',
 'mathematical_and_computer_sciences',
 'chemistry_and_materials',
 'physics',
 'astronautics',
 'social_and_information_sciences',
 'general',
 'aeronautics',
 'geosciences',
 'engineering',
 'life_sciences']

In [6]:
test_str = """Geology (from the Ancient Greek γῆ, gē ("earth") and -λoγία, -logia, ("study of", "discourse")[1][2]) is an earth science concerned with the solid Earth, the rocks of which it is composed, and the processes by which they change over time. Geology can also refer to the study of the solid features of any terrestrial planet or natural satellite such as Mars or the Moon. Modern geology significantly overlaps all other earth sciences, including hydrology and the atmospheric sciences, and so is treated as one major aspect of integrated earth system science and planetary science."""

In [7]:
test_str = """	GeoLab is a geological laboratory and test bed designed for conducting geoscience activities during NASA's analog missions. Scientists at NASA's Johnson Space Center built GeoLab as part of a technology project to aid the development of science operational concepts and to test a variety of analytical tools that could be developed for future planetary surface missions. It is integrated into NASA's Deep Space Habitat (DSH), an exploration habitat test article (Figure 2). As a prototype workstation, GeoLab provides a high fidelity facility for analog mission crewmembers to perform preliminary examination and characterization of geologic samples and communicate their findings to supporting scientists."""

In [8]:
test_str = """Stable rotating semi-rigid bodies were demonstrated analytically, and verified in flights such as Explorer 1 and ATS-5 satellites. The problem arises from the two potential orientations which the final spin vector can take after large angle reorientation from minor to major axis, i.e., along the positive or negative axis of the maximum inertia. Reorientation of a satellite initially spinning about the minor axis using an energy dissipation device may require that the final spin orientation be controlled. Examples of possible applications are the Apogee Motor Assembly with Paired Satellites (AMAPS) configuration, where proper orientation of the thruster is required; and reorientation of ATS-5, where the spin sensitive nature of the despin device (yo-yo mechanism) requires that the final spin vector point is a specified direction."""

In [19]:
# term = terms[10]
predictions = {}
for term in terms:
    print(term)
    indir = 'experiments/general/store/{}/tf'.format(term)
    data_dir = 'experiments/general/store/{}/tf'.format(term)
    files = os.listdir(indir)
    r = re.compile('model.ckpt-[0-9]+.index')
    checkpoints = list(filter(r.match, files))
    values = [int(re.search(r'\d+', checkpoint).group()) for checkpoint in checkpoints]
    i = values.index(max(values))
    ckpt = checkpoints[i]
    init_checkpoint = '{}/{}'.format(indir, ckpt)
    print(init_checkpoint)

    tpu_cluster_resolver = None
    master = None
    run_config = tf.contrib.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      master=master,
      model_dir=output_dir,
      save_checkpoints_steps=save_checkpoints_steps,
      tpu_config=tf.contrib.tpu.TPUConfig(
          iterations_per_loop=iterations_per_loop,
          num_shards=num_tpu_cores,
          per_host_input_for_training=is_per_host))

    model_fn = model_fn_builder(
      bert_config=bert_config,
      num_labels=len(label_list),
      init_checkpoint=init_checkpoint,
      learning_rate=learning_rate,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      use_tpu=False,
      use_one_hot_embeddings=False)

    # If TPU is not available, this will fall back to normal Estimator on CPU
    # or GPU.
    estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=False,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=train_batch_size,
      eval_batch_size=eval_batch_size,
      predict_batch_size=predict_batch_size)

    tokenizer = tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

    # predict_examples = processor.get_test_examples(data_dir)[0:10]

    # make own examples for prediction
    lines = [[], ['0', test_str]]
    predict_examples = processor._create_examples(lines, 'test')
    print(predict_examples[0].text_a)
    predict_file = os.path.join(output_dir, "predict.tf_record")

    max_seq_length = 128
    label_list = ["0", "1"]
    file_based_convert_examples_to_features(predict_examples, label_list,
                                            max_seq_length, tokenizer,
                                            predict_file)

    predict_batch_size = 8
    tf.logging.info("***** Running prediction*****")
    tf.logging.info("  Num examples = %d", len(predict_examples))
    tf.logging.info("  Batch size = %d", predict_batch_size)

    predict_drop_remainder = False
    predict_input_fn = file_based_input_fn_builder(
        input_file=predict_file,
        seq_length=max_seq_length,
        is_training=False,
        drop_remainder=predict_drop_remainder)

    result = estimator.predict(input_fn=predict_input_fn)

    output_predict_file = os.path.join(output_dir, "test_results.tsv")
    with tf.gfile.GFile(output_predict_file, "w") as writer:
        tf.logging.info("***** Predict results *****")
        results = list(result)
        for prediction in results:
            output_line = "\t".join(str(class_probability) for class_probability in prediction) + "\n"
            writer.write(output_line)
    predictions[term] = prediction

space_sciences
experiments/general/store/space_sciences/tf/model.ckpt-12504.index
INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_cluster': None, '_num_ps_replicas': 0, '_task_type': 'worker', '_device_fn': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': 1000, '_model_dir': 'experiments/test', '_is_chief': True, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_master': '', '_log_step_count_steps': None, '_service': None, '_train_distribute': None, '_num_worker_replicas': 1, '_protocol': None, '_evaluation_master': '', '_experimental_distribute': None, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_task_id': 0, '_tf_random_seed': None, '_save_checkpoints_secs': N

INFO:tensorflow:  name = bert/encoder/layer_1/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_1/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/query/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/query/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/key/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/key/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/value/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_2/attention/self/value/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/dense/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_2/attention/output/LayerNorm/beta:0, sh

INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_7/intermediate/dense/kernel:0, shape = (768, 3072)
INFO:tensorflow:  name = bert/encoder/layer_7/intermediate/dense/bias:0, shape = (3072,)
INFO:tensorflow:  name = bert/encoder/layer_7/output/dense/kernel:0, shape = (3072, 768)
INFO:tensorflow:  name = bert/encoder/layer_7/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_7/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_7/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/query/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/query/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_8/attention/self/key/kernel:0, shape = (768, 768)

INFO:tensorflow:_TPUContext: eval_on_tpu True
Stable rotating semi-rigid bodies were demonstrated analytically, and verified in flights such as Explorer 1 and ATS-5 satellites. The problem arises from the two potential orientations which the final spin vector can take after large angle reorientation from minor to major axis, i.e., along the positive or negative axis of the maximum inertia. Reorientation of a satellite initially spinning about the minor axis using an energy dissipation device may require that the final spin orientation be controlled. Examples of possible applications are the Apogee Motor Assembly with Paired Satellites (AMAPS) configuration, where proper orientation of the thruster is required; and reorientation of ATS-5, where the spin sensitive nature of the despin device (yo-yo mechanism) requires that the final spin vector point is a specified direction.
INFO:tensorflow:Writing example 0 of 1
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-1
INFO:tensorfl

INFO:tensorflow:  name = bert/encoder/layer_2/output/dense/kernel:0, shape = (3072, 768)
INFO:tensorflow:  name = bert/encoder/layer_2/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_2/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_2/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/query/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/query/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/key/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/key/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/value/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/value/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_3/attention/output/dense/kernel:0, shape = (768, 768)
INFO

...

INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/query/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/query/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dens

INFO:tensorflow:  name = bert/encoder/layer_9/intermediate/dense/kernel:0, shape = (768, 3072)
INFO:tensorflow:  name = bert/encoder/layer_9/intermediate/dense/bias:0, shape = (3072,)
INFO:tensorflow:  name = bert/encoder/layer_9/output/dense/kernel:0, shape = (3072, 768)
INFO:tensorflow:  name = bert/encoder/layer_9/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_9/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_9/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/query/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/query/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/key/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/key/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/value/kernel:0, shape = (768, 768)
INF

INFO:tensorflow:label: 0 (id = 0)
INFO:tensorflow:***** Running prediction*****
INFO:tensorflow:  Num examples = 1
INFO:tensorflow:  Batch size = 8
INFO:tensorflow:***** Predict results *****
INFO:tensorflow:Could not find trained model in model_dir: experiments/test, running initialization to predict.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
Tensor("loss/one_hot:0", shape=(?, 2), dtype=float32)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (28996, 768)
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768)
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768)
INFO:tensorflow:  name = bert/embe

INFO:tensorflow:  name = bert/encoder/layer_4/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_4/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/query/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/query/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/key/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/key/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/value/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/value/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_5/attention/output/dense/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_5/attention/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_5/attention/output/LayerNorm/beta:0, sh

INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_10/intermediate/dense/kernel:0, shape = (768, 3072)
INFO:tensorflow:  name = bert/encoder/layer_10/intermediate/dense/bias:0, shape = (3072,)
INFO:tensorflow:  name = bert/encoder/layer_10/output/dense/kernel:0, shape = (3072, 768)
INFO:tensorflow:  name = bert/encoder/layer_10/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_10/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_10/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_11/attention/self/query/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_11/attention/self/query/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_11/attention/self/key/kernel:0, shape =

In [20]:
import pandas as pd
df = pd.DataFrame(predictions).T

In [21]:
test_str

'Stable rotating semi-rigid bodies were demonstrated analytically, and verified in flights such as Explorer 1 and ATS-5 satellites. The problem arises from the two potential orientations which the final spin vector can take after large angle reorientation from minor to major axis, i.e., along the positive or negative axis of the maximum inertia. Reorientation of a satellite initially spinning about the minor axis using an energy dissipation device may require that the final spin orientation be controlled. Examples of possible applications are the Apogee Motor Assembly with Paired Satellites (AMAPS) configuration, where proper orientation of the thruster is required; and reorientation of ATS-5, where the spin sensitive nature of the despin device (yo-yo mechanism) requires that the final spin vector point is a specified direction.'

In [22]:
df.sort_values(1, ascending=False)

,0,1
life_sciences,0.399026,0.600974
mathematical_and_computer_sciences,0.478768,0.521232
aeronautics,0.487018,0.512982
astronautics,0.507567,0.492433
social_and_information_sciences,0.510576,0.489424
physics,0.512559,0.487441
geosciences,0.537557,0.462443
space_sciences,0.560743,0.439257
chemistry_and_materials,0.563414,0.436586
engineering,0.564560,0.435440
